# Creating Folds & Preprocessing

We will be using Kaggle Kernels to test XGBoost and CatBoost models using GPU and using these local notebooks to test LightGBM models with CPU. We want our cross-validation scheme to be consistent across both so we define it here, save it using `feather` and upload it to Kaggle.

The original data is very large and takes a long time to load. We use some techniques to save memory and also store the data more efficiently. 

In [1]:
# Global variables for testing changes to this notebook quickly
FOLD_SEED = 0
MIN_FOLDS = 3
MAX_FOLDS = 6

In [2]:
import os
import warnings
import numpy as np
import pandas as pd
import pyarrow
import time

# cross validation
from sklearn.model_selection import StratifiedKFold

# display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

# Loading Times

We benchmark the `read_csv` method with our raw data:

In [3]:
%%time 

train = pd.read_csv('../data/train.csv')

Wall time: 9.24 s


In [4]:
%%time

test = pd.read_csv('../data/test.csv')

Wall time: 4.87 s


# Data Size

We also check how large our raw data is in memory:

In [5]:
print("Train:", 
      round(train.memory_usage().sum() / 1024 ** 2, 2), "Mb")
print("Test:", 
      round(test.memory_usage().sum() / 1024 ** 2, 2), "Mb")

Train: 877.0 Mb
Test: 448.02 Mb


# Reduce Memory Usage

We use a helper function to cast the numerical variables (all variables in this months dataset are numerical) to their lowest possible subtype. This idea was adapted from this [Kaggle notebook](https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro). 

In [6]:
# Creates a copy of the original data
def reduce_memory_usage(data, verbose=True):
    df = data.copy()
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col, dtype in df.dtypes.iteritems():
        if dtype.name.startswith('int'):
            df[col] = pd.to_numeric(data[col], downcast ='integer')
        elif dtype.name.startswith('float'):
            df[col] = pd.to_numeric(data[col], downcast ='float')
            if np.max(df[col] - data[col]) > 1:
                df[col] = data[col]
        
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [7]:
%%time

train_new = reduce_memory_usage(train)
temp = train - train_new
print("Maximal Difference:", np.max(temp.max()))

Mem. usage decreased to 483.26 Mb (44.9% reduction)
Maximal Difference: 0.0003906250003637979
Wall time: 16.4 s


In [8]:
%%time

test_new = reduce_memory_usage(test)
temp = test - test_new
print("Maximal Difference:", np.max(temp.max()))

Mem. usage decreased to 248.48 Mb (44.5% reduction)
Maximal Difference: 0.0003906250003637979
Wall time: 8.52 s


# Stratified K-Fold

We use `StratifiedKFold` to define our cross-validation scheme.

In [9]:
# Create Folds
for NUM_FOLDS in range(MIN_FOLDS,MAX_FOLDS+1):
    train_new[f"{NUM_FOLDS}fold"] = -1
    kf = StratifiedKFold(NUM_FOLDS, shuffle = True, random_state = FOLD_SEED) 
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_new, train_new['claim'])):
        train_new.loc[valid_idx,f"{NUM_FOLDS}fold"] = fold
        
# check output
train_new.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim,3fold,4fold,5fold,6fold
0,0,0.10859,0.004314,-37.566002,0.017364,0.28915,-10.25100,135.119995,168900.0,3.992400e+14,86.488998,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300049,8.6331,4.0607,26.867001,-1.180000,10961.000000,1.5397,135.320007,-1.49650,440.079987,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400024,414.290009,3.5392,1888.000000,0.968930,18.388000,-0.001583,7.7059,5.9325,0.025693,4.5604,0.61122,10.7950,0.341930,0.235010,NaN,5237.700195,1.296100,163.660004,0.403780,0.188600,-0.001446,-0.35416,6.6432,0.30534,0.514020,1.907300e+09,29.861000,0.965010,1797.199951,72.178001,108.620003,1.979900,1.2907,0.99519,1.3228,827.340027,7.779900e+14,4.129900e+10,0.006994,6.9835,43956.000000,1978.199951,5.5084,-0.001081,6.1244,1.231800e+11,275.920013,5308500.0,1704.000000,5.022400e+10,53.397999,-2.201200,6871.000000,3.8862,-0.00558,5252.100098,166.690002,1.60740,0.665340,7768.899902,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.471001,8214.099609,0.288010,0.097826,0.001071,1.412400e+09,0.11093,-12.228000,1.7482,1.90960,-7.11570,4378.799805,1.2096,8.613400e+14,140.100006,1.01770,1,2,3,4,5
1,1,0.10090,0.299610,11822.000000,0.276500,0.45970,-0.83733,1721.900024,119810.0,3.874100e+15,9953.599609,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284000,6.1872,1.0419,4.640400,31.877001,123620.000000,1.3951,125.809998,1.19890,136.449997,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100006,844.820007,1.4680,4726.500000,0.915380,-1.532100,0.982600,7.1112,2.0797,0.042321,4.2523,0.41871,5.4499,0.012737,0.386470,7.3082,283.209991,-0.925520,140.800003,0.247390,-0.001656,-0.000975,-0.22629,2.4246,0.77147,0.011613,1.803700e+09,64.603996,0.262650,4455.000000,78.338997,745.510010,2.906900,1.4826,1.00510,1.4974,84.445999,3.505600e+15,2.242300e+09,0.896300,4.6749,17713.000000,9003.099609,-4.3546,0.254100,6.9191,1.832400e+11,9.651000,32800.0,1480.599976,2.300600e+10,44.050999,205.690002,4295.299805,13.3880,0.46843,754.609985,83.233002,1.18900,29.549999,7343.700195,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.720001,34.341999,0.340300,0.143370,0.049276,1.903200e+09,0.97673,-56.757999,4.1684,0.34808,4.14200,913.229980,1.2464,7.575100e+15,1861.000000,0.28359,0,1,1,2,2
2,2,0.17803,-0.006980,907.270020,0.272140,0.45948,0.17327,2298.000000,360650.0,1.224500e+13,15827.000000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.872999,7.5463,1.9967,1.952600,817.760010,-2948.699951,2.0054,1.682600,1.19680,74.624001,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840027,NaN,1.3561,3063.399902,0.086232,16.106001,0.001481,11.4760,5.3430,0.012162,4.1018,-0.88270,8.1228,-0.676690,0.337700,-1.0732,4097.000000,13.458000,159.240005,0.322300,0.560090,0.000455,-0.16083,3.5753,0.60970,0.028301,5.271300e+08,14.454000,0.115490,14605.000000,36.992001,-9.639100,64.266998,NaN,0.99278,2.5891,430.399994,-4.453500e+13,5.144900e+12,0.099591,6.5516,1887.500000,43319.000000,4.3931,0.260260,6.1052,1.013300e+11,357.269989,1476600.0,90.845001,1.306200e+09,2.373100,391.369995,2965.300049,NaN,0.49459,43.523998,138.520004,1.10790,0.919480,47.915001,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.486000,-83.148003,0.084881,0.032222,0.001668,1.436500e+07,0.20102,-5.768800,1.2042,0.26290,8.13120,45119.000000,1.1764,3.218100e+14,3838.199951,0.40690,1,1,1,2,2
3,3,0.15236,0.007259,780.099976,0.025179,0.51947,7

# Saving Output

We want to save our output in a format that is easy to load and retains our memory usage savings. To this end we use the feather format in pandas.

In [10]:
path_train = '../data/train.feather'
path_test = '../data/test.feather'

# save data
train_new.to_feather(path_train)
test_new.to_feather(path_test)

# Sanity Checks

We check the following:

1. Speed at which we can load `.feather` files
2. Our `.feather` data is equivalent to the original data

## Loading Times

In [11]:
%%time 

# reload data (for testing purposes)
train_df = pd.read_feather(path_train)
test_df = pd.read_feather(path_test)

Wall time: 928 ms


## Data Equivalence

In [12]:
# Check train data types are preserved
types_1 = [train_new[x].dtype for x in train.columns]
types_2 = [train_df[x].dtype for x in train.columns]

assert types_1 == types_2

# Check test data types are preserved
types_1 = [test_new[x].dtype for x in test_new.columns]
types_2 = [test_df[x].dtype for x in test_df.columns]

assert types_1 == types_2

# Find the difference between the data
temp = train_new - train_df
print("Largest Difference (Train):", temp.max().max())

# Find the difference between the data
temp = test_new - test_df
print("Largest Difference (Test):", temp.max().max())

Largest Difference (Train): 0.0
Largest Difference (Test): 0.0
